# 모두닥 크롤링 모듈

In [2]:
!pip install tqdm
!pip install selenium

In [6]:
from tqdm import notebook
import pandas as pd    # csv
from selenium.webdriver.chrome.options import *
from bs4 import *
import time
import re
import base64    # encoder
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

ImportError: cannot import name 'Mapping' from 'collections' (c:\Users\Dr.song\AppData\Local\Programs\Python\Python310\lib\collections\__init__.py)

In [7]:
# 버전 체크

print(wb.__version__)

4.3.0


# 병원 리스트들 뜨는 페이지에서 처리하는 부분

In [28]:
# 현재 페이지 내 병원 박스 가져오는 함수
def get_hospital_boxes():
    element_boxes = driver.find_elements("xpath", "//div[@class='doctor-total-box border-bottom']")
    
    return element_boxes


# 한 페이지에서 병원 url들과 리뷰 갯수 리스트로 가져오는 함수
def get_list_url_and_num_review(hospital_boxes):
    list_data = []
    for hospital_box in hospital_boxes:
        data = [get_hospital_url(hospital_box), get_num_review(hospital_box)]
        list_data.append(data)
        
    return list_data


# 박스에서 리뷰 갯수 가져오는 함수
def get_num_review(element_box):
    num_review = element_box.find_element(By.CLASS_NAME, "hospitals-review-count").text
    num_review = num_review[3:]
    num_review = int(num_review)
    
    return num_review


# 병원 url 가져오는 함수
def get_hospital_url(element_box):
    temp = element_box.find_element(By.TAG_NAME, "a")
    hospital_url = temp.get_attribute("href")
    
    return hospital_url


# 병원 리스트 페이지에서에서 해당 페이지로 넘어가는 함수
def move_in_hospital_list_page(hospital_kind_query, hospital_kind, page):
    page_url = hospital_kind_query + hospital_kind + "&page=" + str(page)
    driver.get(page_url)


# 리뷰 페이지에서 처리하는 부분

In [31]:
# 병원 이름 가져오는 함수
def get_hospital_name():
    hospital_name = driver.find_element(By.CLASS_NAME, "hospital-doctor-name-box").text
    
    return hospital_name


# 리뷰 박스들 가져오는 함수
def get_review_boxes():
    element_boxes = driver.find_elements("xpath", "//div[@class='review-in-detail-text-box-content col-8 pl-5']")
    
    return element_boxes


# 리뷰 박스에서 받은 치료 가져오는 함수
def get_therapy(element_box):
    therapy = element_box.find_element(By.CLASS_NAME, "title-box").text
    therapy = therapy[8:]
    
    return therapy


# 리뷰 박스에서 리뷰 텍스트 가져오는 함수
def get_review(element_box):
    review = element_box.find_element(By.CLASS_NAME, "content-box").text
    review = clean_text(review)

    return review


# 리뷰 페이지에서 해당 페이지로 넘어가는 함수
def move_in_review_page(current_url, page):
    page_url = current_url +  "?page=" + str(page)
    driver.get(page_url)


# 병원 분과 페이지에서 총 몇 페이지까지 있는지 구하는 함수
def get_total_page():
    move_to_last_page()
    time.sleep(1)
    total_page = get_page_count()
    driver.back()
    time.sleep(1)
    
    return total_page


# 끝 페이지 이동하는 함수
def move_to_last_page():
    driver.find_element('xpath', '/html/body/div[3]/div/div/div[1]/div[3]/nav/ul/li[4]/a').click()

    
# 현재 페이지 몇인지 구하는 함수
def get_page_count():
    page_count = []
    
    for text in driver.current_url[::-1]:
        if text == '=':
            break
        page_count.append(text)
    page_count.reverse()
    page_count = int(''.join(page_count))
    
    return page_count


In [3]:
# 로그인
def login(id, password):
    driver.find_element('xpath', '/html/body/nav/div/div/ul[2]/ul/li[3]/a').click() #id 속성으로 접근

    user_email = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[1]')
    user_email.send_keys(id)
    user_password = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[2]')
    user_password.send_keys(password)

    # 로그인 버튼 클릭
    driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/button').click()
    time.sleep(3)


# 텍스트 전처리
def clean_text(text):
    cleaned_text = re.sub('[\n]', ' ', text)
    
    return cleaned_text

# 크롤링 데이터 전처리

In [4]:
# 엑셀 파일 리뷰 부분 전처리하는 함수(.xlsx 제외하고 입력)
def do_preprocess(file_name):
    df = pd.read_excel(file_name + ".xlsx")
    not_using_text_list = ["한줄평", "병원리뷰", r"[0-9]."]
    
    for i in range(len(df)):
        temp = df.loc[i, '리뷰']
        temp = preprocess(temp)
        
        for not_using_text in not_using_text_list:
            temp = remove_text(temp, not_using_text)
            
        df.loc[i, '리뷰'] = temp
        
    df.to_excel(file_name + "전처리" + ".xlsx")

    
# 텍스트 전처리하는 함수
def preprocess(text):
    text= re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", ".", text)
    text = re.sub(r"[\`\~\@\#\$\%\^\&\*\(\)\-\_\=\+\<\>\;\:\'\"\[\]\{\}\\\/]", " ", text)
    text = re.sub(r"[^가-힣A-Za-z0-9\.\?\!]", " ", text)
    text = re.sub(r"\s*[\.]+", ".", text)
    text = re.sub(r"\s*[\?]+", "?", text)
    text = re.sub(r"\s*[\!]+", "!", text)
    text = re.sub(r"\s+", " ", text)
    text.strip()
    
    return text


# 불용어 제거하는 함수
def remove_text(original_text, not_using_text):
    result = re.sub(not_using_text, "", original_text)
    
    return result

In [4]:
# 모두닥 페이지 접속 후 로그인

# 모두닥 페이지 url
modoodoc_url = "https://www.modoodoc.com/"

## 크롬 드라이버 실행
#  wb.Chrome() = 컴퓨터가 크롬창을 씀
driver = wb.Chrome()

# 모두닥 페이지 이동
driver.get(modoodoc_url)

# 로그인
id = 'random1234@random.com'
password = 'random1234'
login(id, password)

NameError: name 'wb' is not defined

In [ ]:
# 병원 분과 종류
hospital_kind_list = ['치과', '피부과', '성형외과', '산부인과', '정신건강의학과', '비뇨기과','재활의학과', '외과', '소아과', '가정의학과', '한의원']

# 모두닥 병원 접근 쿼리
hospital_kind_query = "https://www.modoodoc.com/hospitals/?search_query="


for hospital_kind in hospital_kind_list:
    writer = pd.ExcelWriter(f'모두닥({hospital_kind}).xlsx', engine='xlsxwriter')
    count = 0
    
    driver.get(hospital_kind_query + hospital_kind)
    time.sleep(1)
    hospital_total_page = get_total_page()
    time.sleep(1)
    
    
    data = []
    data_unit = []

    for i in notebook.tqdm(range(1, hospital_total_page + 1)):
        try:
            if (count > 25000):
                break

            move_in_hospital_list_page(hospital_kind_query, hospital_kind, i)
            time.sleep(1)

            hospital_boxes = get_hospital_boxes()
            list_url_and_num_review = get_list_url_and_num_review(hospital_boxes)


            for url, num_review in list_url_and_num_review:
                try:
#                     if (count > 25000):
#                                     break

                    if (num_review == 0):
                        continue

                    num_review_page = (num_review // 10) + 1

        #             driver.get(url)
        #             time.sleep(1)

        #             current_url = driver.current_url

                    for page in range(1, num_review_page+1):
                        try:
#                             if (count > 25000):
#                                         break

                            move_in_review_page(url, page)
                            time.sleep(1)

                            hospital_name = get_hospital_name()
                            review_boxes = get_review_boxes()

                            for review_box in review_boxes:
                                try:
                                    if (count > 25000):
                                        break

                                    therapy = get_therapy(review_box)
                                    review = get_review(review_box)

                                    data_unit = [hospital_kind, hospital_name, therapy, review]
                                    data.append(data_unit)
                                    count += 1
                                    
                                except:
                                    pass
                        except:
                            pass
                except:
                    pass
        except:
            pass

                    
    # 데이터 프레임 생성 및 데이터 입력
    df = pd.DataFrame(columns = ['분과', '병원명', '받은 치료', '리뷰'])
    num = len(data)

    for i in range(num):
        df.loc[i]=data[i]

   
    df.to_excel(f'모두닥({hospital_kind}).xlsx', sheet_name = hospital_kind)

  0%|          | 0/430 [00:00<?, ?it/s]

  0%|          | 0/176 [00:00<?, ?it/s]